Student Participant number: 05112003 
(Please note that this isn't the one attributed to me by UCL, but should be a unique string, as suggested by Gabriel Brostow's email)

# Practical 7 - Part 2A
This second half of the lab explores the geometry of a single camera. In 2B the goal is to use a set of correspondance points to estimate a transformation matrix from a plane's 3D space to camera space and use that matrix to project some other points into camera space.

In this section, we'll work on building two components that we need for 2B, a method to estimate that transformation and a method that can project points into camera image space.

First we'll tackle the projection method, `projectiveCamera`. We want to find the image space coordinates, `XImCart`, of a set of 3D world coordinates, `XCart`, given a camera intrinsics matrix `K` and an extrinsics matrix `T`.

The second component is a method to estimate a Eucledian transformation, `TEst`, that takes us from a plane's 3D coordinate space to 3D camera space by utilizing a given set of points in camera image space, `XImCart`, and a set of corresponding points in world space, `XCart`. Essentially we want to compute the extrinsics matrix we can use in `projectiveCamera`.

Estimating the camera pose will involve calculating a homography, so you'll need to copy over your functions from part 1A/1B in the space provided.

## Import libraries 

In [2]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

In [3]:
def projectiveCamera(K,T,XCart):
    ##TODO
    # The goal of this function is to project points in XCart through projective camera
    # defined by intrinsic matrix K and extrinsic matrix T. In essence, this function takes a set of points 
    # in 3D world space, XCart, and projects them into camera image space by applying the extrinsic matrix T 
    # and then applying the intrinsic matrix K.
    # 
    # There are three steps.
    # 1) Move from world space to camera space. 
    #            camera space points = extrinsics T * world space points 
    #
    # 2) Applying the intrinsics matrix to the camera space points after normalizing
    #           homogeneous image space points = K * normalized camera space points
    # 
    # 3) Move to image space cartesian points from image space homogeneous points, involves a 
    # normalization using the third row.
    
    # TODO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    n = XCart.shape[1]
    XHom = np.vstack((XCart, np.ones((1, n))))

    # TODO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    xCamHom = T @ XHom 

    # TODO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    xCamCart = xCamHom[:3, :] / xCamHom[3, :]

    # TODO: Move points to image coordinates xImHom by applying intrinsic matrix
    xImHom = K @ xCamCart

    # TODO: Convert points back to Cartesian coordinates xImCart
    XImCart = xImHom[:2, :] / xImHom[2, :]

    return XImCart


## Camera Projection

First we'll write up the function that can take us from 3D world space, `XCart`, to camera image space, `XImCart`, using an extrinsics matrix `T` and an intrinsics matrix `K` that are provided. The previous block houses this function.

The result here is the cartesian image space point coordinates, `XImCart`, of the 3D points `XCart`. If `XCart` represents a box in the world then we now know where the box's vertices would land in image space.

To verify that your solution is correct please compare your image space points to those in the comment.

Once they match, move on to the next bit - estimating a transformation! 

In [4]:
# We assume that the camera intrinsics matrix K is known and has values:
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are five known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
# T houses a rotation matrix and a translation matrix. The last row is for homogeneous point calculation.


# TODO: Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera"
XImCart = projectiveCamera(K,T,XCart)

print(XImCart)
# Should be around:
# [267.4170882  230.95045427 531.42492013 482.36049098 378.77537982]
# [396.26814909 288.11435494 237.83410247 358.39940241 329.44079538]

[[267.4170882  230.95045427 531.42492013 482.36049098 378.77537982]
 [396.26814909 288.11435494 237.83410247 358.39940241 329.44079538]]


### You've implemented both of these functions in 1A and 1B already, so feel free to copy them in here. You'll need them for this next part.

In [6]:
def solveAXEqualsZero(A):
    # TODO: Write this routine - it should solve Ah = 0. You can do this using SVD. Consult your notes! 
    # Hint: SVD will be involved. 
  
    # We compute the SVD of A
    U, S, Vt = np.linalg.svd(A)
    
    # The vector h is the last column of V corresponding to the smallest singular value
    h = Vt[-1, :]
    
    # We normalize h if we want the last element to be 1.
    if abs(h[-1]) > 1e-12:
        h = h / h[-1]
    return h

In [7]:
def calcBestHomography(pts1Cart, pts2Cart):
    # This function should apply the direct linear transform (DLT) algorithm to calculate the best 
    # homography that maps the cartesian points in pts1Cart to their corresonding matching cartesian poitns 
    # in pts2Cart.
    
    # This function calls solveAXEqualsZero. Make sure you are wary of how to reshape h into a 3 by 3 matrix. 

    n_points = pts1Cart.shape[1]
    
    # TODO: replace this:
    H = np.identity(3)

    # TODO: 
    # First convert points into homogeneous representation
    # Hint: we've done this before  in the skeleton code we provide.
    pts1Hom = np.vstack((pts1Cart, np.ones((1, n_points))))
    pts2Hom = np.vstack((pts2Cart, np.ones((1, n_points))))

    # TODO: 
    # Then construct the matrix A, size (n_points * 2, 9)
    # Consult the notes!
    A = np.zeros((2 * n_points, 9))

    for i in range(n_points):
        x,  y  = pts1Hom[0, i],  pts1Hom[1, i]
        x_p, y_p = pts2Hom[0, i], pts2Hom[1, i]
        A[2*i, :]   = [0, 0, 0, -x, -y, -1,  y_p*x,  y_p*y,  y_p]
        A[2*i+1, :] = [x, y, 1,  0,  0,  0, -x_p*x, -x_p*y, -x_p]

    # TODO: 
    # Solve Ah = 0 using solveAXEqualsZero and get h.
    h = solveAXEqualsZero(A)

    # TODO: 
    # Reshape h into the matrix H, values of h go first into rows of H
    H = h.reshape(3, 3)
    
    return H

In [8]:
# Read the next cell first for context!

def estimatePlanePose(XImCart,XCart,K):
    # The goal of this function is to estimate the pose of a plane relative to camera (extrinsic matrix)
    # given points in image space xImCart, points in 3D world space XCart, and an intrinsics matrix K.

    # TODO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    n = XImCart.shape[1]
    XImHom = np.vstack((XImCart, np.ones((1, n))))
    
    # TODO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    K_inv = np.linalg.inv(K)
    XCamHom = K_inv @ XImHom
    xCamNorm = XCamHom[:2, :] / XCamHom[2, :]
    
    # TODO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    XCart2D = XCart[:2, :]
    H_est = calcBestHomography(XCart2D, xCamNorm)
          
    # TODO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD. NOTE: You do not need to transpose v from linalg.svd    
    h1 = H_est[:, 0]
    h2 = H_est[:, 1]
    h3 = H_est[:, 2]
    Q = np.column_stack((h1, h2))
    U, S, Vt = np.linalg.svd(Q)
    R12 = U[:, :2] @ Vt
    r1 = R12[:, 0]
    r2 = R12[:, 1]

    # TODO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    r3 = np.cross(r1, r2)
    R = np.column_stack((r1, r2, r3))
        
    # TODO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    if np.linalg.det(R) < 0:
        R[:, 2] = -R[:, 2]
    
    # TODO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    norm_h1 = np.linalg.norm(h1)
    norm_h2 = np.linalg.norm(h2)
    k = (norm_h1 + norm_h2) / 2.0
    t = h3 / k
    
    # TODO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[2] < 0:
        R[:, 0] = -R[:, 0]
        R[:, 1] = -R[:, 1]
        t = -t
            
    # TODO: Assemble transformation into matrix form
    T = np.eye(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = t
    
    return T 


## Now the juicy bit: Estimating an Extrinsics Matrix, T

The problem: We are given an instrinsics matrix `K`, a set of 3D world points `XCart`, and a set of corresponding image space coordinates in `XImCart`. `K` and `XCart` have already been defined a few cells back and you've calculated `XImCart` by virtue of the exercise you've completed with camera projection. What we don't have is an extrinsics matrix, `T`. We need to estimate this and you'll need to fill in `estimatePlanePose` and return `TEst`.

Again you can start by negating the noise we add to XImCart to make sure you're on the right track.

In [15]:
# TODO: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
sigma = 1.0
XImCartNoisy = XImCart + np.random.normal(loc=0.0, scale=sigma, size=XImCart.shape)

# TODO: Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function estimatePlanePose
TEst = estimatePlanePose(XImCartNoisy,XCart,K)

# If you have got this correct, TEst should closely resemble the groundtruth, T.
np.set_printoptions(precision=3)
print(TEst)
print("\n")
print(T)


[[ 9.866e-01 -5.698e-02  1.530e-01  4.594e+01]
 [-1.589e-01 -5.516e-01  8.188e-01  6.971e+01]
 [ 3.776e-02 -8.321e-01 -5.533e-01  5.002e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


[[ 9.851e-01 -4.920e-02  1.619e-01  4.600e+01]
 [-1.623e-01 -5.520e-01  8.181e-01  7.000e+01]
 [ 4.900e-02 -8.324e-01 -5.518e-01  5.009e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


The result above suggests a correct implementation, as TEst closely resemble the ground truth T. 